<a href="https://colab.research.google.com/github/MahdiZaman/Spring2019-CAP5610/blob/master/hw1p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###HW1 P1

import numpy as np
from keras.datasets import mnist
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()


Using TensorFlow backend.


In [0]:
# Preparing the Train and Test Images (Adding a column of 1s for biases)
nfeatures = 28*28
nclasses = 1
ntrainsamples = train_images_original.shape[0] #60000
ntestsamples = test_images_original.shape[0]   #10000

#normalizing inputs
train_images = train_images_original.reshape((ntrainsamples, nfeatures))
train_images = train_images.astype('float32') / 255 
test_images = test_images_original.reshape((ntestsamples, nfeatures))
test_images = test_images.astype('float32') / 255

#adding a column of 1s for bias calculation
train_images_b = np.c_[np.ones((ntrainsamples, 1)), train_images]
test_images_b = np.c_[np.ones((ntestsamples,1)), test_images]

# # One hot encoding for train and test labels
train_labels_bin = np.eye(10)[train_labels_original]
test_labels_bin = np.eye(10)[test_labels_original]

In [0]:
def sigmoid(z):
    a = 1/(1+ np.exp(-z))
    return a

In [0]:
def sigdif(a):
    return a *(1-a)

In [0]:
def deriv(x, y, a, minibatch):
    temp = a-y
    dz = ((temp) * sigdif(a)) / minibatch
    dw = x.T.dot(dz)
    return dw

In [0]:
def forwardprop(x, weight):
    xw = x.dot(weight)
    A = sigmoid(xw)
    return A

In [0]:
def mse_loss(y, y_hat):
    l = 0.5*np.mean((y_hat-y)**2)
    return l

In [15]:
## Training the model

epochs = 10 #number of epochs
minibatchsize = 40 #number of samples in minibatch
lr = 0.01 #Learning Rate

#initializing Weights and Bias(Bias added as a column of 1 on the left in weight array)

total_w=np.zeros((785,10))

for label in range(10):

    #initial_weight = np.random.randn(nfeatures+1, nclasses)
    initial_weight = np.zeros((nfeatures+1, nclasses))
    weight = initial_weight
  
    #weight_path_mgd = []
    #weight_path_mgd.append(weight)

    for epoch in range(epochs):
        #shuffling the samples
        m = np.random.permutation(ntrainsamples)
        train_images = train_images_b[m]
        train_labels = train_labels_bin[m]

        #minibatch SGD
        for i in range(0, ntrainsamples, minibatchsize):
            train_images_m = train_images[i : i+minibatchsize]
            train_labels_m = train_labels[i : i+minibatchsize]

            A = forwardprop(train_images_m, weight)
            
            train_labels_l = train_labels_m[:,label]
            train_labels_l = train_labels_l.reshape((minibatchsize,1))

            loss = mse_loss(train_labels_l, A)
            
# # #             # Updating Weights and Biases


            dw = deriv(train_images_m, train_labels_l, A , minibatchsize)
            weight = weight - lr*dw
            
            #weight_path_mgd.append(weight)
    total_w[:,label]=weight.reshape(785,)    
    
#      weight_path_mgd = np.array([])
#      weight_path_mgd = np.c_[weight_path_mgd , weight.reshape((785,1))]
#weight_path_mgd_array = np.asarray(weight_path_mgd)
print(total_w.shape)

(785, 10)


In [16]:
print(loss)
print(total_w.shape)

0.01296814382278208
(785, 10)


In [20]:
z = test_images_b.dot(total_w)
a = sigmoid(z)
labels = np.argmax(a,axis=1)
comp = (labels==test_labels_original)
acc = sum(comp)/(test_labels_original.shape[0])
print("Accuracy is {}".format(acc))

Accuracy is 0.8841
